In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras import layers, callbacks, models, optimizers
from tensorflow.keras.utils import to_categorical, plot_model
import yaml
from attrdict import AttrDict
from os import path
from tqdm import tqdm
from statistics import mean, stdev
from sklearn import svm

Filepaths

In [ ]:
dataset_dir = path.join('..', '..', '..', '..', 'Data', 'UCI HAR Dataset')

Reading hyperparameters

In [ ]:
with open('signalDataLSTMhparams.yaml') as params_file:
    config = yaml.full_load(params_file)
    hparams = AttrDict(config)

Reading Dataset

In [ ]:
sensor_types = ['body_acc', 'body_gyro', 'total_acc']
axis = ['x', 'y', 'z']

In [ ]:
def get_dataset(col_type, dataset_type):
    if col_type == 'X':
        dataset = list()
        for sensor in sensor_types:
            for direction in axis:
                filepath = path.join(dataset_dir, dataset_type, '_'.join([sensor, direction, dataset_type])+'.csv')
                dataset.append(pd.read_csv(filepath, delim_whitespace=True, header=None))
        return np.array(np.dstack(dataset))
    else:
        y = pd.read_csv(path.join(dataset_dir, dataset_type, 'y_' + dataset_type + '.txt'), header=None)
        return to_categorical(y-1)
        # return y

In [ ]:
X_train = get_dataset('X', 'train')
y_train = get_dataset('y', 'train')

In [ ]:
X_test = get_dataset('X', 'test')
y_test = get_dataset('y', 'test')

# Building Model

In [ ]:
def build_model(hparams, verbose=True):
    model = models.Sequential()
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(hparams.neuron_count, activation="tanh"), input_shape=(128, hparams.n_features)))
    model.add(layers.Dropout(0.28))
    model.add(layers.Dense(100))
    model.add(layers.Dense(hparams.output_neuron, activation="softmax"))
    model.compile(optimizer="sgd", loss="kullback_leibler_divergence", metrics=["accuracy"])
    if verbose:
        print(model.summary())
    return model

In [ ]:
model = build_model(hparams)

In [ ]:
plot_model(model, 'model.png', show_shapes=True, show_dtype=True, show_layer_names=True)

In [ ]:
es = callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)
model.fit(X_train, y_train, epochs=35, verbose=True,  validation_split=0.1, batch_size=32)

# Testing Model

In [ ]:
model.evaluate([X_test, means_test], y_test, verbose=0)

In [ ]:
results = model.predict(X_test)
evaluation = list()
for i in range(len(results)):
    evaluation.append(str(np.argmax(results[i])+1) + '-' + str(np.argmax(y_test[i])+1))
evals = pd.DataFrame(evaluation, columns=['eval'])
evals.value_counts()

# Stress Test

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=0, patience=5)
accuracies = list()
for i in tqdm(range(10)):
    model = build_model(hparams, verbose=False)
    model.fit(X_train, y_train, epochs=25, verbose=False,  validation_split=0.1, batch_size=64, callbacks=[es])
    result = model.evaluate(X_test, y_test)[1]
    accuracies.append(result)
    print(result)

In [ ]:
print("max: ", max(accuracies) * 100, "\nmin: ", min(accuracies) * 100, "\nmean: ", mean(accuracies), "\nstd: ", stdev(accuracies))